In [122]:
import tkinter as tk
from tkinter import ttk
from tkinter import filedialog as fd
from tkinter import filedialog as fd
from numpy.core.defchararray import startswith
from numpy.core.numeric import outer
import pyxlsb
import pandas as pd
import numpy as np
import subprocess
import numpy as np




win = tk.Tk()
win.title("Đối chiếu Paid list")
#-------------------------------------------------------------Declare global variables ------------------------------

paidListFile = ''
paidListFileUseCol=['Paid dated','Note','Currency','Amount']
paidListFileUseColDataType={'Paid dated': np.datetime64, 'Note': object , 'Currency' : object, 'Amount': np.float64 }

accFile = ''
accFileUseCol=['Ngày chứng từ','Số đơn', 'Số đơn HT', 'Số đơn ĐT','Loại tiền', 'TK Nợ', 'TK Có', 'Số tiền', 'Số tiền VND']
accFileUseColDataType={'Ngày chứng từ':np.datetime64,'TK Nợ':object,'Loại tiền': object, 'TK Có':object, 'Số tiền':np.float64, 'Số tiền VND':np.float64}
savingFolder = ''
#radPremiumOrOthers=''
dfPaidList=pd.DataFrame
dfAcc=pd.DataFrame
dfAccFinal=pd.DataFrame
dfPaidListFinal=pd.DataFrame
dfPaidListGroupBy=pd.DataFrame

#------------------------------------------------------------- Widget event handle------------------------------------


def click_select_cpc():
    global paidListFile
    paidListFile = fd.askopenfilename()


def click_select_acc():
    global accFile
    accFile = fd.askopenfilename()


def click_save_file():
    global savingFolder
    savingFolder = fd.askdirectory()
    #print(saving_folder)

def button_open_saved_file_click():
    subprocess.Popen([f"{savingFolder}/{savingFileName.get()}.csv"], shell=True)
    win.destroy()

#def rad_select_premium_click():
 #   global premium_or_claim
  #  premium_or_claim = radPremiumOrOthers.get() # return 'premium'


#def rad_select_claim_click():
 #   global premium_or_claim
  #  premium_or_claim = radPremiumOrOthers.get() # return 'claim'

def return_policy(soDonHachToan, soDonDoiTru, soDon, taiKhoanNo, taiKhoanCo):
    if len(soDonHachToan) < 2 and len(soDonDoiTru) < 2:
        return soDon
    elif len(soDonHachToan) < 2 and len(soDon) < 2:
        return soDonDoiTru
    elif len(soDon) < 2 and len(soDonDoiTru) < 2:
        return soDonHachToan
    elif taiKhoanCo.startswith('112'): # TK no -> 'So don DT', TK co -> 'So don HT'
        if len(soDonDoiTru)>2:
            return soDonDoiTru
        else:
            return soDon
    elif taiKhoanNo.startswith('112'): # TK no -> 'So don DT', TK co -> 'So don HT'
        if len(soDonHachToan)>2:
            return soDonHachToan
        else:
            return soDon
    else:
        return


def return_account(taiKhoanNo,taiKhoanCo):
    if taiKhoanNo.startswith('112'):
        return taiKhoanNo
    elif taiKhoanCo.startswith('112'):
        return taiKhoanCo
    else:
        return 'TaiKhoanKhac'


def return_account_du(taiKhoanNo,taiKhoanCo):
    if taiKhoanNo.startswith('112'):
        return taiKhoanCo
    elif taiKhoanCo.startswith('112'):
        return taiKhoanNo
    else:
        return 'TaiKhoanKhac'

def return_amount(taiKHoanNo,taiKhoanCo,soTien):
    if taiKHoanNo.startswith('112'):
        return soTien
    elif taiKhoanCo.startswith('112'):
        return -soTien
    else:
        return 0



def click_execute():
    from tkinter import messagebox as msg
    global dfAccFinal
    global dfPaidListFinal
    global dfPaidList
    global dfAcc
    global dfPaidListGroupBy
    name=['No','Paid dated','Client Name','Amount','Currency','Type','Bank/Cash','Note','Place number']
    try:
        if paidListFile.endswith('xlsb'):
            dfPaidList = pd.read_excel(paidListFile, usecols=paidListFileUseCol,sheet_name='Claim paid', skiprows=3,header=None,names=name,engine='pyxlsb')
        else:
            dfPaidList = pd.read_excel(paidListFile, usecols=paidListFileUseCol,sheet_name='Claim paid', skiprows=3,header=None,names=name)
                  
        if accFile.endswith('xlsb'):
            dfAcc = pd.read_excel(accFile,usecols=accFileUseCol,skiprows=6,dtype=accFileUseColDataType, engine='pyxlsb')
        else:
            dfAcc = pd.read_excel(accFile,usecols=accFileUseCol,skiprows=6,dtype=accFileUseColDataType)
    except:
        msg.showerror("Error", "Only Excel files or CSV files are supported")
        pass

    values={'Số đơn':'0', 'Số đơn HT':'0','Số đơn ĐT':'0'}
    dfAcc.fillna(value=values, inplace=True)
    dfAcc['Policy']=dfAcc.apply(lambda x: return_policy(x['Số đơn'], x['Số đơn HT'], x['Số đơn ĐT'], x['TK Nợ'], x['TK Có']), axis=1)
    dfAcc['Tai khoan']=dfAcc.apply(lambda x: return_account(x['TK Nợ'], x['TK Có']), axis=1)  
    dfAcc['Tai khoan doi ung']=dfAcc.apply(lambda x: return_account_du(x['TK Nợ'], x['TK Có']), axis=1)  
    dfAcc['So tien NT']=dfAcc.apply(lambda x: return_amount(x['TK Nợ'], x['TK Có'],x['Số tiền']), axis=1) 
    dfAcc['So tien VND']=dfAcc.apply(lambda x: return_amount(x['TK Nợ'], x['TK Có'],x['Số tiền VND']), axis=1) 
    dfAccFinal=dfAcc.loc[(dfAcc['Tai khoan']!='TaiKhoanKhac') & (dfAcc['Tai khoan doi ung'].str.startswith('624')),['Ngày chứng từ','Policy','Loại tiền','So tien NT']] 
    dfAccFinalGroupby=dfAccFinal.groupby(['Ngày chứng từ','Policy','Loại tiền'],as_index=False)['So tien NT'].sum()
    a=np.datetime64('2000-01-01')
    values={'Paid dated':a ,'Note': '0','Currency':'0','Amount':0}
    dfPaidList=dfPaidList.fillna(value=values)
    dfPaidListGroupBy=dfPaidList.groupby(['Paid dated','Note','Currency'],as_index=False)['Amount'].sum()
    dfmerge=dfPaidListGroupBy.merge(dfAccFinalGroupby,how='outer',left_on=['Paid dated','Note','Currency'],right_on=['Ngày chứng từ','Policy','Loại tiền'],suffixes=['PL','ACC'])



    #-------------------------------------------------------------------------------------


    #df_merge_all.to_csv(f"{saving_folder}/{saving_file_name.get()}.csv", encoding="utf-8-sig", index=False)
    # df_merge.to_csv(c+r'/result5.csv',encoding="utf-8-sig",index=False)
    msg.showinfo("Information", "Đã hoàn thành")
    win.destroy()  # Close window after execution


# ----------------------------------------------------------------- Widget------------------------------------------------

#radPremiumOrOthers = tk.StringVar()


#rad_select_premium = ttk.Radiobutton(win, text='Premium Paid', width=30, value='premium', variable=radPremiumOrOthers, command=rad_select_premium_click)
#rad_select_premium.grid(column=0, row=0)

#rad_select_claim = ttk.Radiobutton(win, text='Claim paid', width=30, value='claim', variable=radPremiumOrOthers, command=rad_select_claim_click)
#rad_select_claim.grid(column=1, row=0)

ttk.Label(win, text="Chọn file Paid list", width=20).grid(column=0, row=1)
button1 = ttk.Button(win, text="Paid list", command=click_select_cpc)
button1.grid(column=1, row=1)

ttk.Label(win, text="Chọn file CTGS", width=20).grid(column=0, row=2)
button2 = ttk.Button(win, text="CTGS", command=click_select_acc)
button2.grid(column=1, row=2)

ttk.Label(win, text="Chọn thư mục lưu file", width=20).grid(column=0, row=3)
button2 = ttk.Button(win, text="Folder", command=click_save_file)
button2.grid(column=1, row=3)

ttk.Label(win, text="Nhập tên file", width=20).grid(column=0, row=4)
savingFileName = ttk.Entry(win, width=10)
savingFileName.grid(column=1, row=4)

ttk.Label(win, text="Xử lý số liệu", width=20).grid(column=0, row=5)
button3 = ttk.Button(win, text="Run", command=click_execute)
button3.grid(column=1, row=5)

label_open_saved_file = ttk.Label(win, text="Open saved file", width=30).grid(column=0, row=6)
button_open_saved_file = ttk.Button(win, text="Open saved file", command=button_open_saved_file_click)
button_open_saved_file.grid(column=1, row=6)




win.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.8_3.8.1776.0_x64__qbz5n2kfra8p0\lib\tkinter\__init__.py", line 1883, in __call__
    return self.func(*args)
  File "<ipython-input-122-1b000511ba27>", line 137, in click_execute
    dfAcc.fillna(value=values, inplace=True)
TypeError: fillna() missing 1 required positional argument: 'self'


In [59]:
values={'Paid dated':a ,'Note': '0','Currency':'0','Amount':0}
dfPaidList=dfPaidList.fillna(value=values)

In [60]:
dfPaidList

,Paid dated,Amount,Currency,Note
0,2020-09-01,2.400000e+04,VND,hoa hong dai ly t7/2020 inv 0004140
1,2020-09-01,1.725500e+08,VND,UIC Hoàn phí bảo hiểm PAR190195HYD.CEN01
2,2020-09-01,2.466281e+06,VND,tien boi thuong WCI200163HYD.003.CL01.BT.3
3,2020-09-01,4.630497e+06,VND,tien boi thuong MCI2001HYD.06-002.CL01.BT.2
4,2020-09-01,5.564955e+07,VND,"UIC Giảm phí bảo hiểm theo điều khoản ""No Clai..."
...,...,...,...,...
260,2000-01-01,0.000000e+00,0,0
261,2000-01-01,0.000000e+00,0,0
262,2000-01-01,2.151927e+10,VND,0
263,2000-01-01,1.174838e+06,USD,0


In [55]:
import numpy as np
a=np.datetime64('2000-01-01')
dfPaidListFinal['Ngay']=dfPaidListFinal['Ngay'].fillna(a)

TypeError: only integer scalar arrays can be converted to a scalar index

In [25]:
b.head()

,Ngay
0,2020-09-01
1,2020-09-01
2,2020-09-01
3,2020-09-01
4,2020-09-01


In [27]:
import numpy as np
a=np.datetime64('2000-01-01')
b=b.fillna(a)

In [28]:
b

,Ngay
0,2020-09-01
1,2020-09-01
2,2020-09-01
3,2020-09-01
4,2020-09-01
...,...
260,2000-01-01
261,2000-01-01
262,2000-01-01
263,2000-01-01


In [37]:

c=['Paid dated','Note','Currency','Amount']
d={'Paid dated': np.datetime64, 'Note': object , 'Currency' : object, 'Amount': np.float64 }
name=['No','Paid dated','Client Name','Amount','Currency','Type','Bank/Cash','Note','Place number']
dfa=pd.read_excel("C:/Users/tuandq/Desktop/TEST/2020_09_30 - Claim, commission and other expense paid list.xlsx",sheet_name='Claim paid',skiprows=3,usecols=c,dtype=d,header=None, names=name)

In [39]:
dfa=dfa.loc[:,['Paid dated','Note','Currency','Amount']]

In [43]:
import numpy as np
a=np.datetime64('2000-01-01')
dfa['Paid dated']=dfa['Paid dated'].fillna(a)

In [54]:
dfa.dtypes

Paid dated    datetime64[ns]
Note                  object
Currency              object
Amount               float64
dtype: object

In [58]:
dfPaidList['Paid dated']=dfPaidList['Paid dated'].fillna(a)

In [66]:
dfPaidListFinal=dfPaidList.loc[:,['Paid dated','Note','Currency','Amount']].reset_index(drop=True)


In [67]:
dfPaidListFinal

,Paid dated,Note,Currency,Amount
0,2020-09-01,hoa hong dai ly t7/2020 inv 0004140,VND,2.400000e+04
1,2020-09-01,UIC Hoàn phí bảo hiểm PAR190195HYD.CEN01,VND,1.725500e+08
2,2020-09-01,tien boi thuong WCI200163HYD.003.CL01.BT.3,VND,2.466281e+06
3,2020-09-01,tien boi thuong MCI2001HYD.06-002.CL01.BT.2,VND,4.630497e+06
4,2020-09-01,"UIC Giảm phí bảo hiểm theo điều khoản ""No Clai...",VND,5.564955e+07
...,...,...,...,...
260,2000-01-01,0,0,0.000000e+00
261,2000-01-01,0,0,0.000000e+00
262,2000-01-01,0,VND,2.151927e+10
263,2000-01-01,0,USD,1.174838e+06


In [68]:
dfPaidListFinal.columns=[['Ngay','So don','Loai tien','So tien']]

In [72]:

dfAccF

Ngày chứng từ    datetime64[ns]
Policy                   object
Loại tiền                object
So tien NT              float64
dtype: object

In [69]:
dfPaidList.groupby(['Ngay','So don','Loai tien']).sum()

TypeError: only integer scalar arrays can be converted to a scalar index

In [ ]:



dfmerge=dfPaidList.merge(dfAccFinalGroupby,how='outer',left_on=['Ngay','So don','Loai tien'],right_on=['Ngày chứng từ','Policy','Loại tiền'],suffixes=['PL','ACC'])
dfmerge.to_csv("D:/merge.csv")

In [73]:
 dfAccFinalGroupby=dfAccFinal.groupby(['Ngày chứng từ','Policy','Loại tiền']).sum()

In [74]:
dfPaidList.groupby(['Paid dated','Note','Currency']).sum()

Amount
Paid dated Note                                          Currency              
2000-01-01 0                                             0         0.000000e+00
                                                         JPY       0.000000e+00
                                                         USD       1.174838e+06
                                                         VND       2.151927e+10
           HCI190008HYD.CL19.BT.1                        0         7.220971e+07
...                                                                         ...
2020-09-29 UIC TT TIEN THAY TRONG, MUC MAY IN, INV004892 VND       5.689000e+06
2020-09-30 BOI THUONG PAI200001HGDEN.CLAIM.200526.CL01   VND       1.255812e+07
           HOAN PHI MDL200080HYDEN02, INV00894           VND       9.246575e+06
           PAYMENT FOR REINSURANCE                       USD       3.545590e+03
           TT HOA HONG DAI LY T8/2020                    VND       5.076362e+07

[256 rows x 1 columns]

In [75]:
dfPaidListGroupBy=dfPaidList.groupby(['Paid dated','Note','Currency','Amount']).sum()
dfPaidListGroupBy.columns=[['Ngay','So don','Loai tien','So tien']]

    

dfPaidListGroupBy=dfPaidListFinal.groupby(['Ngay','So don','Loai tien']).sum()
dfmerge=dfPaidList.merge(dfAccFinalGroupby,how='outer',left_on=['Ngay','So don','Loai tien'],right_on=['Ngày chứng từ','Policy','Loại tiền'],suffixes=['PL','ACC'])
dfmerge.to_csv("D:/merge.csv")


ValueError: Length mismatch: Expected axis has 0 elements, new values have 4 elements

In [87]:
dfPaidListGroupBy=dfPaidList.groupby(['Paid dated','Note','Currency'],as_index=False)['Amount'].sum()

In [91]:
dfAccFinal

,Ngày chứng từ,Policy,Loại tiền,So tien NT
280,2020-01-22,0,VND,-28520000.0
290,2020-01-22,PAR190013SBL.C,VND,-10500000.0
291,2020-01-15,PAR180585SYD.C,VND,-16416758.0
479,2020-02-12,TT PHI DICH VU EPOLICY T1/2020 INV0000135,VND,-12298000.0
480,2020-02-12,TT PHI NGHIEN CUU THI TRUONG KH NHAT,VND,-880000.0
...,...,...,...,...
55010,2020-08-27,Inv0001982_2.694.531,VND,-408545.0
55011,2020-08-27,Inv0001982_2.694.531,VND,-184091.0
55012,2020-08-27,Inv0001982_2.694.531,VND,-184091.0
55013,2020-08-27,Inv0001982_2.694.531,VND,-184091.0


In [82]:
dfPaidListGroupBy.to_csv("D:/groupby.csv")

In [84]:
dfAccFinalGroupby.to_csv('D:/acgroupby.csv')

KeyError: 'Column not found: So tien'

In [101]:
values={'Số đơn':'0', 'Số đơn HT':'0','Số đơn ĐT':'0'}
dfAcc.fillna(value=values, inplace=True)
dfAcc['Policy']=dfAcc.apply(lambda x: return_policy(x['Số đơn'], x['Số đơn HT'], x['Số đơn ĐT'], x['TK Nợ'], x['TK Có']), axis=1)
dfAcc['Tai khoan']=dfAcc.apply(lambda x: return_account(x['TK Nợ'], x['TK Có']), axis=1)  
dfAcc['Tai khoan doi ung']=dfAcc.apply(lambda x: return_account_du(x['TK Nợ'], x['TK Có']), axis=1)  
dfAcc['So tien NT']=dfAcc.apply(lambda x: return_amount(x['TK Nợ'], x['TK Có'],x['Số tiền']), axis=1) 
dfAcc['So tien VND']=dfAcc.apply(lambda x: return_amount(x['TK Nợ'], x['TK Có'],x['Số tiền VND']), axis=1) 
dfAccFinal=dfAcc.loc[(dfAcc['Tai khoan']!='TaiKhoanKhac') & (dfAcc['Tai khoan doi ung'].str.startswith('624')),['Ngày chứng từ','Policy','Loại tiền','So tien NT']] 
dfAccFinalGroupby=dfAccFinal.groupby(['Ngày chứng từ','Policy','Loại tiền'],as_index=False)['So tien NT'].sum()
a=np.datetime64('2000-01-01')
values={'Paid dated':a ,'Note': '0','Currency':'0','Amount':0}
dfPaidList=dfPaidList.fillna(value=values)
dfPaidListGroupBy=dfPaidList.groupby(['Paid dated','Note','Currency'],as_index=False)['Amount'].sum()
#dfPaidListGroupBy.columns=[['Ngay','So don','Loai tien','So tien']]
dfmerge=dfPaidListGroupBy.merge(dfAccFinalGroupby,how='outer',left_on=['Paid dated','Note','Currency'],right_on=['Ngày chứng từ','Policy','Loại tiền'],suffixes=['PL','ACC'])
dfmerge.to_csv("D:/merge.csv")


In [100]:
dfAccFinalGroupby.index

Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
            ...
            4177, 4178, 4179, 4180, 4181, 4182, 4183, 4184, 4185, 4186],
           dtype='int64', length=4187)

In [99]:
dfPaidListGroupBy.index

Int64Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
            ...
            246, 247, 248, 249, 250, 251, 252, 253, 254, 255],
           dtype='int64', length=256)

In [105]:
dfAcc=dfAcc.loc[dfAcc['TK Có'].str.startswith('112')]

In [108]:
a=dfAcc['TK Nợ'].unique()

In [109]:
a

array(['335282', '141111', '642722', '133118', '112211', '112111',
       '153111', '133111', '642131', '335281', '138888', '642811',
       '642852', '642711', '642831', '213511', '642511', '133116',
       '642741', '642813', '642132', '642851', '642733', '642713',
       '642111', '331121', '333532', '642163', '333514', '333114',
       '333534', '331311', '642721', '642211', '642715', '338888',
       '338711', '624194', '133126', '642123', '624183', '333121',
       '333512', '331211', '642853', '642151', '642141', '333861',
       '333111', '333511', '331141', '642121', '642112', '642125',
       '642126', '333411', '333531', '338813', '133112', '642712',
       '642814', '624184', '624187', '642841', '642731', '642133',
       '642732', '624112', '133121', '112261', '112114', '642221',
       '624181', '133117', '642771', '642762', '112117', '642884',
       '635111', '624191', '334111', '642124', '624188', '112121',
       '335111', '335211', '642524', '642168', '331511', '3312

In [140]:
import tkinter as tk
from tkinter import ttk
from tkinter import filedialog as fd
from tkinter import filedialog as fd
from numpy.core.defchararray import startswith
from numpy.core.numeric import outer
import pyxlsb
import pandas as pd
import numpy as np
import subprocess
import numpy as np





win = tk.Tk()
win.title("Đối chiếu Paid list")
#-------------------------------------------------------------Declare global variables ------------------------------

paidListFile = ''
paidListFileUseCol=['Paid dated','Số hồ sơ BT/Số đơn','Currency','Số tiền']
paidListFileUseColDataType={'Paid dated': np.datetime64, 'Note': object , 'Currency' : object, 'Amount': np.float64 }

accFile = ''
accFileUseCol=['Ngày chứng từ','Số đơn', 'Số đơn HT', 'Số đơn ĐT','Loại tiền', 'TK Nợ', 'TK Có', 'Số tiền', 'Số tiền VND']
accFileUseColDataType={'Ngày chứng từ':np.datetime64,'TK Nợ':object,'Loại tiền': object, 'TK Có':object, 'Số tiền':np.float64, 'Số tiền VND':np.float64}
savingFolder = ''
savingFileName=''
#radPremiumOrOthers=''
dfPaidList=pd.DataFrame
dfAcc=pd.DataFrame
dfAccFinal=pd.DataFrame
dfPaidListFinal=pd.DataFrame
dfPaidListGroupBy=pd.DataFrame
dfMerge=pd.DataFrame

fullOrLeft=''

#------------------------------------------------------------- Widget event handle------------------------------------


def click_select_cpc():
    global paidListFile
    paidListFile = fd.askopenfilename()


def click_select_acc():
    global accFile
    accFile = fd.askopenfilename()


def click_save_file():
    global savingFolder
    savingFolder = fd.askdirectory()
    #print(saving_folder)

def button_open_saved_file_click():
    subprocess.Popen([f"{savingFolder}/{savingFileName.get()}.csv"], shell=True)
    win.destroy()

def rad_select_full_click():
    global fullOrLeft
    fullOrLeft = RadFullOrLeft.get() # return 'full'


def rad_select_left_click():
    global fullOrLeft
    fullOrLeft = RadFullOrLeft.get() # return 'left'

def return_policy(soDonHachToan, soDonDoiTru, soDon, taiKhoanNo, taiKhoanCo):
    if len(soDonHachToan) < 2 and len(soDonDoiTru) < 2:
        return soDon
    elif len(soDonHachToan) < 2 and len(soDon) < 2:
        return soDonDoiTru
    elif len(soDon) < 2 and len(soDonDoiTru) < 2:
        return soDonHachToan
    elif taiKhoanCo.startswith('112'): # TK no -> 'So don DT', TK co -> 'So don HT'
        if len(soDonDoiTru)>2:
            return soDonDoiTru
        else:
            return soDon
    elif taiKhoanNo.startswith('112'): # TK no -> 'So don DT', TK co -> 'So don HT'
        if len(soDonHachToan)>2:
            return soDonHachToan
        else:
            return soDon
    else:
        return


def return_account(taiKhoanNo,taiKhoanCo):
    if taiKhoanNo.startswith('112'):
        return taiKhoanNo
    elif taiKhoanCo.startswith('112'):
        return taiKhoanCo
    else:
        return 'TaiKhoanKhac'


def return_account_du(taiKhoanNo,taiKhoanCo):
    if taiKhoanNo.startswith('112'):
        return taiKhoanCo
    elif taiKhoanCo.startswith('112'):
        return taiKhoanNo
    else:
        return 'TaiKhoanKhac'

def return_amount(taiKHoanNo,taiKhoanCo,soTien):
    if taiKHoanNo.startswith('112'):
        return soTien
    elif taiKhoanCo.startswith('112'):
        return -soTien
    else:
        return 0



def click_execute():
    from tkinter import messagebox as msg
    global dfAccFinal
    global dfPaidListFinal
    global dfPaidList
    global dfAcc
    global dfPaidListGroupBy
    global dfMerge
    global paidListFile
    global accFile
    global savingFolder
    global savingFileName

    #name=['No','Paid dated','Client Name','Amount','Currency','Type','Bank/Cash','Note','Số tiền','Place number','Số hồ sơ BT/Số đơn','Kỳ TT','Mã KH','Tài khoản']
    try:
        if paidListFile.endswith('xlsb'):
            dfPaidList = pd.read_excel(paidListFile, usecols=paidListFileUseCol,sheet_name='Claim paid', skiprows=1,engine='pyxlsb')
        else:
            dfPaidList = pd.read_excel(paidListFile, usecols=paidListFileUseCol,sheet_name='Claim paid', skiprows=1)
                  
        if accFile.endswith('xlsb'):
            dfAcc = pd.read_excel(accFile,usecols=accFileUseCol,skiprows=6,dtype=accFileUseColDataType, engine='pyxlsb')
        else:
            dfAcc = pd.read_excel(accFile,usecols=accFileUseCol,skiprows=6,dtype=accFileUseColDataType)
    except:
        msg.showerror("Error", "Only Excel files or CSV files are supported")
        pass

    values={'Số đơn':'0', 'Số đơn HT':'0','Số đơn ĐT':'0'}
    dfAcc=dfAcc.fillna(value=values)
    dfAcc=dfAcc.loc[dfAcc['TK Có'].str.startswith('112')]
    dfAcc['Policy']=dfAcc.apply(lambda x: return_policy(x['Số đơn'], x['Số đơn HT'], x['Số đơn ĐT'], x['TK Nợ'], x['TK Có']), axis=1)
    dfAcc['Tai khoan']=dfAcc.apply(lambda x: return_account(x['TK Nợ'], x['TK Có']), axis=1)  
    dfAcc['Tai khoan doi ung']=dfAcc.apply(lambda x: return_account_du(x['TK Nợ'], x['TK Có']), axis=1)  
    dfAcc['So tien NT']=dfAcc.apply(lambda x: return_amount(x['TK Nợ'], x['TK Có'],x['Số tiền']), axis=1) 
    dfAcc['So tien VND']=dfAcc.apply(lambda x: return_amount(x['TK Nợ'], x['TK Có'],x['Số tiền VND']), axis=1) 
    dfAccFinal=dfAcc.loc[(dfAcc['Tai khoan']!='TaiKhoanKhac') & (dfAcc['Tai khoan doi ung'].str.startswith(('624','331','333','131','133','138','338'))),['Ngày chứng từ','Policy','Loại tiền','So tien NT']] 
    dfAccFinalGroupby=dfAccFinal.groupby(['Ngày chứng từ','Policy','Loại tiền'],as_index=False)['So tien NT'].sum()
    a=np.datetime64('2000-01-01')
    values={'Paid dated':a ,'Số hồ sơ BT/Số đơn': '0','Currency':'0','Số tiền':0}
    dfPaidList=dfPaidList.fillna(value=values)
    dfPaidListGroupBy=dfPaidList.groupby(['Paid dated','Số hồ sơ BT/Số đơn','Currency'],as_index=False)['Số tiền'].sum()
    if fullOrLeft=='full':
        dfMerge=dfPaidListGroupBy.merge(dfAccFinalGroupby,how='outer',left_on=['Paid dated','Số hồ sơ BT/Số đơn','Currency'],right_on=['Ngày chứng từ','Policy','Loại tiền'],suffixes=['PL','ACC'])
    else:
        dfMerge=dfPaidListGroupBy.merge(dfAccFinalGroupby,how='left',left_on=['Paid dated','Số hồ sơ BT/Số đơn','Currency'],right_on=['Ngày chứng từ','Policy','Loại tiền'],suffixes=['PL','ACC'])
    dfMerge['Chenh lech']=dfMerge['Số tiền']-dfMerge['So tien NT']
    dfMerge.to_csv(f"{savingFolder}/{savingFileName.get()}.csv", encoding="utf-8-sig", index=False)
    msg.showinfo("Information", "Đã hoàn thành")
    subprocess.Popen([f"{savingFolder}/{savingFileName.get()}.csv"], shell=True)
    
    paidListFile = ''
    accFile = ''
    savingFolder = ''
    savingFileName=''
    #radPremiumOrOthers=''
    #dfPaidList=pd.DataFrame
    #dfAcc=pd.DataFrame
    #dfAccFinal=pd.DataFrame
    #dfPaidListFinal=pd.DataFrame
    #dfPaidListGroupBy=pd.DataFrame


# ----------------------------------------------------------------- Widget------------------------------------------------

RadFullOrLeft = tk.StringVar()


rad_select_full = ttk.Radiobutton(win, text='Full', width=30, value='full', variable=RadFullOrLeft, command=rad_select_full_click)
rad_select_full.grid(column=0, row=0)

rad_select_claim = ttk.Radiobutton(win, text='Left', width=30, value='left', variable=RadFullOrLeft, command=rad_select_left_click)
rad_select_claim.grid(column=1, row=0)

ttk.Label(win, text="Chọn file Paid list", width=20).grid(column=0, row=1)
button1 = ttk.Button(win, text="Paid list", width=20, command=click_select_cpc)
button1.grid(column=1, row=1)

ttk.Label(win, text="Chọn file CTGS", width=20).grid(column=0, row=2)
button2 = ttk.Button(win, text="CTGS", width=20, command=click_select_acc)
button2.grid(column=1, row=2)

ttk.Label(win, text="Chọn thư mục lưu file", width=20).grid(column=0, row=3)
button2 = ttk.Button(win, text="Folder", width=20, command=click_save_file)
button2.grid(column=1, row=3)

ttk.Label(win, text="Nhập tên file", width=20).grid(column=0, row=4)
savingFileName = ttk.Entry(win, width=20)
savingFileName.grid(column=1, row=4)

ttk.Label(win, text="Xử lý số liệu", width=20).grid(column=0, row=5)
button3 = ttk.Button(win, text="Run", width=20, command=click_execute)
button3.grid(column=1, row=5)

label_open_saved_file = ttk.Label(win, text="Mở file", width=20).grid(column=0, row=6)
button_open_saved_file = ttk.Button(win, text="Mở file", width=20, command=button_open_saved_file_click)
button_open_saved_file.grid(column=1, row=6)




win.mainloop()

WARNING *** file size (923330) not 512 + multiple of sector size (512)


In [137]:
dfPaidList

pandas.core.frame.DataFrame

In [133]:
dfPaidList

,No,Paid\n dated,Client Name,Amount,Currency,Type,Bank/Cash,Note,Place number,Số tiền,Số hồ sơ BT/Số đơn,Kỳ TT,Mã KH,Tài khoản,"Hạch toán nợ ghi số dương, hạch toán có ghi số âm",Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18
0,1.0,2,3,4.000000e+00,5,6.0,7,8,9,10.0,11,12,13,14.0,NaN,NaN,NaN,NaN,NaN
1,1.0,2020-09-01 00:00:00,CONG TY TNHH THUONG MAI ACS VIET NAM,2.400000e+04,VND,NaN,VID,hoa hong dai ly t7/2020 inv 0004140,Commission,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2.0,2020-09-01 00:00:00,CONG TY TNHH SHINDENGEN VIET NAM,1.725500e+08,VND,NaN,VID,UIC Hoàn phí bảo hiểm PAR190195HYD.CEN01,Refund,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3.0,2020-09-01 00:00:00,DUONG THI THUY TIEN,2.466281e+06,VND,NaN,VID,tien boi thuong WCI200163HYD.003.CL01.BT.3,Claim,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4.0,2020-09-01 00:00:00,"CANON VIETNAM CO., LTD",4.630497e+06,VND,NaN,VID,tien boi thuong MCI2001HYD.06-002.CL01.BT.2,Claim,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
550,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
551,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
552,NaN,NaN,Sub total:,2.250722e+10,VND,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
553,NaN,NaN,NaN,1.171292e+06,USD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [126]:
dfAcc

,Ngày chứng từ,Số đơn,Số đơn HT,Số đơn ĐT,TK Nợ,TK Có,Số tiền,Loại tiền,Số tiền VND
0,2020-09-29,NaN,NaN,VND49.901.801,338888,112131,4.990180e+07,VND,4.990180e+07
1,2020-09-29,NaN,NaN,NaN,112111,112134,1.600000e+09,VND,1.600000e+09
2,2020-09-29,NaN,NaN,NaN,642722,112131,1.562760e+05,VND,1.562760e+05
3,2020-09-29,NaN,NaN,NaN,133116,112131,1.562700e+04,VND,1.562700e+04
4,2020-09-29,NaN,350.435.266VND,350.435.266VND,138888,112114,3.504353e+08,VND,3.504353e+08
...,...,...,...,...,...,...,...,...,...
957,2020-09-29,WCI190472SYDEN01,NaN,NaN,624142,331511,3.294758e+06,VND,3.294758e+06
958,2020-09-29,PAR200022HYD.CEN02,NaN,NaN,624142,331511,2.269000e+01,USD,5.282460e+05
959,2020-09-29,FFL190075SYD2020.08EN01,NaN,NaN,331211,112131,6.944090e+05,VND,6.944090e+05
960,2020-09-29,GOL190345SYD.CL01.BT.2,NaN,NaN,624111,111112,3.652400e+06,VND,3.652400e+06


In [117]:
values={'Số đơn':'0', 'Số đơn HT':'0','Số đơn ĐT':'0'}
dfAcc=dfAcc.fillna(value=values)

In [141]:
dfAccGroupByVoucher=dfAccFinal.groupby(['Ngày chứng từ','Policy','Loại tiền'],as_index=False)['Số chứng từ'].transform(lambda x: ','.join(x))

KeyError: 'Column not found: Số chứng từ'

In [142]:
import tkinter as tk
from tkinter import ttk
from tkinter import filedialog as fd
from tkinter import filedialog as fd
from numpy.core.defchararray import startswith
from numpy.core.numeric import outer
import pyxlsb
import pandas as pd
import numpy as np
import subprocess
import numpy as np





win = tk.Tk()
win.title("Đối chiếu Paid list")
#-------------------------------------------------------------Declare global variables ------------------------------

paidListFile = ''
paidListFileUseCol=['Paid dated','Số hồ sơ BT/Số đơn','Currency','Số tiền']
paidListFileUseColDataType={'Paid dated': np.datetime64, 'Note': object , 'Currency' : object, 'Amount': np.float64 }

accFile = ''
accFileUseCol=['Ngày chứng từ','Số chứng từ','Số đơn', 'Số đơn HT', 'Số đơn ĐT','Loại tiền', 'TK Nợ', 'TK Có', 'Số tiền', 'Số tiền VND']
accFileUseColDataType={'Ngày chứng từ':np.datetime64,'TK Nợ':object,'Loại tiền': object, 'TK Có':object, 'Số tiền':np.float64, 'Số tiền VND':np.float64}
savingFolder = ''
savingFileName=''
#radPremiumOrOthers=''
dfPaidList=pd.DataFrame
dfAcc=pd.DataFrame
dfAccFinal=pd.DataFrame
dfPaidListFinal=pd.DataFrame
dfPaidListGroupBy=pd.DataFrame
dfMerge=pd.DataFrame
dfAccGroupByVoucher=pd.DataFrame

fullOrLeft=''

#------------------------------------------------------------- Widget event handle------------------------------------


def click_select_cpc():
    global paidListFile
    paidListFile = fd.askopenfilename()


def click_select_acc():
    global accFile
    accFile = fd.askopenfilename()


def click_save_file():
    global savingFolder
    savingFolder = fd.askdirectory()
    #print(saving_folder)

def button_open_saved_file_click():
    subprocess.Popen([f"{savingFolder}/{savingFileName.get()}.csv"], shell=True)
    win.destroy()

def rad_select_full_click():
    global fullOrLeft
    fullOrLeft = RadFullOrLeft.get() # return 'full'


def rad_select_left_click():
    global fullOrLeft
    fullOrLeft = RadFullOrLeft.get() # return 'left'

def return_policy(soDonHachToan, soDonDoiTru, soDon, taiKhoanNo, taiKhoanCo):
    if len(soDonHachToan) < 2 and len(soDonDoiTru) < 2:
        return soDon
    elif len(soDonHachToan) < 2 and len(soDon) < 2:
        return soDonDoiTru
    elif len(soDon) < 2 and len(soDonDoiTru) < 2:
        return soDonHachToan
    elif taiKhoanCo.startswith('112'): # TK no -> 'So don DT', TK co -> 'So don HT'
        if len(soDonDoiTru)>2:
            return soDonDoiTru
        else:
            return soDon
    elif taiKhoanNo.startswith('112'): # TK no -> 'So don DT', TK co -> 'So don HT'
        if len(soDonHachToan)>2:
            return soDonHachToan
        else:
            return soDon
    else:
        return


def return_account(taiKhoanNo,taiKhoanCo):
    if taiKhoanNo.startswith('112'):
        return taiKhoanNo
    elif taiKhoanCo.startswith('112'):
        return taiKhoanCo
    else:
        return 'TaiKhoanKhac'


def return_account_du(taiKhoanNo,taiKhoanCo):
    if taiKhoanNo.startswith('112'):
        return taiKhoanCo
    elif taiKhoanCo.startswith('112'):
        return taiKhoanNo
    else:
        return 'TaiKhoanKhac'

def return_amount(taiKHoanNo,taiKhoanCo,soTien):
    if taiKHoanNo.startswith('112'):
        return soTien
    elif taiKhoanCo.startswith('112'):
        return -soTien
    else:
        return 0



def click_execute():
    from tkinter import messagebox as msg
    global dfAccFinal
    global dfPaidListFinal
    global dfPaidList
    global dfAcc
    global dfPaidListGroupBy
    global dfAccGroupByVoucher
    global dfMerge
    global paidListFile
    global accFile
    global savingFolder
    global savingFileName

    #name=['No','Paid dated','Client Name','Amount','Currency','Type','Bank/Cash','Note','Số tiền','Place number','Số hồ sơ BT/Số đơn','Kỳ TT','Mã KH','Tài khoản']
    try:
        if paidListFile.endswith('xlsb'):
            dfPaidList = pd.read_excel(paidListFile, usecols=paidListFileUseCol,sheet_name='Claim paid', skiprows=1,engine='pyxlsb')
        else:
            dfPaidList = pd.read_excel(paidListFile, usecols=paidListFileUseCol,sheet_name='Claim paid', skiprows=1)
                  
        if accFile.endswith('xlsb'):
            dfAcc = pd.read_excel(accFile,usecols=accFileUseCol,skiprows=6,dtype=accFileUseColDataType, engine='pyxlsb')
        else:
            dfAcc = pd.read_excel(accFile,usecols=accFileUseCol,skiprows=6,dtype=accFileUseColDataType)
    except:
        msg.showerror("Error", "Only Excel files or CSV files are supported")
        pass

    values={'Số đơn':'0', 'Số đơn HT':'0','Số đơn ĐT':'0'}
    dfAcc=dfAcc.fillna(value=values)
    dfAcc=dfAcc.loc[dfAcc['TK Có'].str.startswith('112')]
    dfAcc['Policy']=dfAcc.apply(lambda x: return_policy(x['Số đơn'], x['Số đơn HT'], x['Số đơn ĐT'], x['TK Nợ'], x['TK Có']), axis=1)
    dfAcc['Tai khoan']=dfAcc.apply(lambda x: return_account(x['TK Nợ'], x['TK Có']), axis=1)  
    dfAcc['Tai khoan doi ung']=dfAcc.apply(lambda x: return_account_du(x['TK Nợ'], x['TK Có']), axis=1)  
    dfAcc['So tien NT']=dfAcc.apply(lambda x: return_amount(x['TK Nợ'], x['TK Có'],x['Số tiền']), axis=1) 
    dfAcc['So tien VND']=dfAcc.apply(lambda x: return_amount(x['TK Nợ'], x['TK Có'],x['Số tiền VND']), axis=1) 
    dfAccFinal=dfAcc.loc[(dfAcc['Tai khoan']!='TaiKhoanKhac') & (dfAcc['Tai khoan doi ung'].str.startswith(('624','331','333','131','133','138','338'))),['Ngày chứng từ','Policy','Loại tiền','So tien NT']] 
    dfAccGroupByVoucher=dfAccFinal.groupby(['Ngày chứng từ','Policy','Loại tiền'],as_index=False)['Số chứng từ'].transform(lambda x: ','.join(x))
    
    dfPaidList=dfPaidList[dfPaidList['Số hồ sơ BT/Số đơn'].notna()]
    a=np.datetime64('2000-01-01')
    values={'Paid dated':a ,'Số hồ sơ BT/Số đơn': '0','Currency':'0','Số tiền':0}
    dfPaidList=dfPaidList.fillna(value=values)
    dfPaidListGroupBy=dfPaidList.groupby(['Paid dated','Số hồ sơ BT/Số đơn','Currency'],as_index=False)['Số tiền'].sum()
    if fullOrLeft=='full':
        dfMerge=dfPaidListGroupBy.merge(dfAccFinalGroupby,how='outer',left_on=['Paid dated','Số hồ sơ BT/Số đơn','Currency'],right_on=['Ngày chứng từ','Policy','Loại tiền'],suffixes=['PL','ACC'])
    else:
        dfMerge=dfPaidListGroupBy.merge(dfAccFinalGroupby,how='left',left_on=['Paid dated','Số hồ sơ BT/Số đơn','Currency'],right_on=['Ngày chứng từ','Policy','Loại tiền'],suffixes=['PL','ACC'])
    dfMerge['Chenh lech']=dfMerge['Số tiền']+dfMerge['So tien NT']
    dfMerge.to_csv(f"{savingFolder}/{savingFileName.get()}.csv", encoding="utf-8-sig", index=False)
    msg.showinfo("Information", "Đã hoàn thành")
    subprocess.Popen([f"{savingFolder}/{savingFileName.get()}.csv"], shell=True)
    
    paidListFile = ''
    accFile = ''
    savingFolder = ''
    savingFileName=''
    #radPremiumOrOthers=''
    #dfPaidList=pd.DataFrame
    #dfAcc=pd.DataFrame
    #dfAccFinal=pd.DataFrame
    #dfPaidListFinal=pd.DataFrame
    #dfPaidListGroupBy=pd.DataFrame


# ----------------------------------------------------------------- Widget------------------------------------------------

RadFullOrLeft = tk.StringVar()


rad_select_full = ttk.Radiobutton(win, text='Full', width=30, value='full', variable=RadFullOrLeft, command=rad_select_full_click)
rad_select_full.grid(column=0, row=0)

rad_select_claim = ttk.Radiobutton(win, text='Left', width=30, value='left', variable=RadFullOrLeft, command=rad_select_left_click)
rad_select_claim.grid(column=1, row=0)

ttk.Label(win, text="Chọn file Paid list", width=20).grid(column=0, row=1)
button1 = ttk.Button(win, text="Paid list", width=20, command=click_select_cpc)
button1.grid(column=1, row=1)

ttk.Label(win, text="Chọn file CTGS", width=20).grid(column=0, row=2)
button2 = ttk.Button(win, text="CTGS", width=20, command=click_select_acc)
button2.grid(column=1, row=2)

ttk.Label(win, text="Chọn thư mục lưu file", width=20).grid(column=0, row=3)
button2 = ttk.Button(win, text="Folder", width=20, command=click_save_file)
button2.grid(column=1, row=3)

ttk.Label(win, text="Nhập tên file", width=20).grid(column=0, row=4)
savingFileName = ttk.Entry(win, width=20)
savingFileName.grid(column=1, row=4)

ttk.Label(win, text="Xử lý số liệu", width=20).grid(column=0, row=5)
button3 = ttk.Button(win, text="Run", width=20, command=click_execute)
button3.grid(column=1, row=5)

label_open_saved_file = ttk.Label(win, text="Mở file", width=20).grid(column=0, row=6)
button_open_saved_file = ttk.Button(win, text="Mở file", width=20, command=button_open_saved_file_click)
button_open_saved_file.grid(column=1, row=6)




win.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.8_3.8.1776.0_x64__qbz5n2kfra8p0\lib\tkinter\__init__.py", line 1883, in __call__
    return self.func(*args)
  File "<ipython-input-142-786e5410f416>", line 158, in click_execute
    dfAccGroupByVoucher=dfAccFinal.groupby(['Ngày chứng từ','Policy','Loại tiền'],as_index=False)['Số chứng từ'].transform(lambda x: ','.join(x))
  File "C:\Users\tuandq\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\pandas\core\groupby\generic.py", line 1613, in __getitem__
    return super().__getitem__(key)
  File "C:\Users\tuandq\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\pandas\core\base.py", line 234, in __getitem__
    raise KeyError(f"Column not found: {key}")
KeyError: 'Column not found: Số chứng từ'


In [143]:
dfAcc

,Số chứng từ,Ngày chứng từ,Số đơn,Số đơn HT,Số đơn ĐT,TK Nợ,TK Có,Số tiền,Loại tiền,Số tiền VND,Policy,Tai khoan,Tai khoan doi ung,So tien NT,So tien VND
0,04.09_20.0104,2020-09-28,0,0,0,642722,112261,1.210000e+02,JPY,2.722500e+04,0,112261,642722,-1.210000e+02,-2.722500e+04
1,04.09_20.0105,2020-09-28,0,0,0,642722,112281,9.100000e-01,EUR,2.523300e+04,0,112281,642722,-9.100000e-01,-2.523300e+04
2,04.09_20.0128,2020-09-29,0,0,VND49.901.801,338888,112131,4.990180e+07,VND,4.990180e+07,VND49.901.801,112131,338888,-4.990180e+07,-4.990180e+07
3,04.09_20.0130,2020-09-29,0,0,0,112111,112134,1.600000e+09,VND,1.600000e+09,0,112111,112134,1.600000e+09,1.600000e+09
4,04.09_20.0154,2020-09-28,TT TIEN MUA QUA TRUNG THU INV 0049465,0,0,642813,112114,2.361600e+07,VND,2.361600e+07,TT TIEN MUA QUA TRUNG THU INV 0049465,112114,642813,-2.361600e+07,-2.361600e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1523,603.09_20.0002,2020-09-28,PAR180005HBF.C.CL01.BT.4,0,0,624112,112114,2.250000e+06,VND,2.250000e+06,PAR180005HBF.C.CL01.BT.4,112114,624112,-2.250000e+06,-2.250000e+06
1525,605.09_20.0046,2020-09-28,WCI180002HYD_HOYA.004.CL01.BT.6,0,0,331111,112114,1.119763e+07,VND,1.119763e+07,WCI180002HYD_HOYA.004.CL01.BT.6,112114,331111,-1.119763e+07,-1.119763e+07
1526,605.09_20.0047,2020-09-28,WCI200002HYD_HOYA.001.CL29.BT.2,0,0,331111,112114,1.840400e+06,VND,1.840400e+06,WCI200002HYD_HOYA.001.CL29.BT.2,112114,331111,-1.840400e+06,-1.840400e+06
1527,605.09_20.0049,2020-09-28,WCI200002HYD_HOYA.001.CL22.BT.2,0,0,331111,112114,9.831000e+05,VND,9.831000e+05,WCI200002HYD_HOYA.001.CL22.BT.2,112114,331111,-9.831000e+05,-9.831000e+05


In [144]:
import tkinter as tk
from tkinter import ttk
from tkinter import filedialog as fd
from tkinter import filedialog as fd
from numpy.core.defchararray import startswith
from numpy.core.numeric import outer
import pyxlsb
import pandas as pd
import numpy as np
import subprocess
import numpy as np





win = tk.Tk()
win.title("Đối chiếu Paid list")
#-------------------------------------------------------------Declare global variables ------------------------------

paidListFile = ''
paidListFileUseCol=['Paid dated','Số hồ sơ BT/Số đơn','Currency','Số tiền']
paidListFileUseColDataType={'Paid dated': np.datetime64, 'Note': object , 'Currency' : object, 'Amount': np.float64 }

accFile = ''
accFileUseCol=['Ngày chứng từ','Số chứng từ','Số đơn', 'Số đơn HT', 'Số đơn ĐT','Loại tiền', 'TK Nợ', 'TK Có', 'Số tiền', 'Số tiền VND']
accFileUseColDataType={'Ngày chứng từ':np.datetime64,'TK Nợ':object,'Loại tiền': object, 'TK Có':object, 'Số tiền':np.float64, 'Số tiền VND':np.float64}
savingFolder = ''
savingFileName=''
#radPremiumOrOthers=''
dfPaidList=pd.DataFrame
dfAcc=pd.DataFrame
dfAccFinal=pd.DataFrame
dfPaidListFinal=pd.DataFrame
dfPaidListGroupBy=pd.DataFrame
dfMerge=pd.DataFrame
dfAccGroupByVoucher=pd.DataFrame
dfAccFinalVoucher=pd.DataFrame

fullOrLeft=''

#------------------------------------------------------------- Widget event handle------------------------------------


def click_select_cpc():
    global paidListFile
    paidListFile = fd.askopenfilename()


def click_select_acc():
    global accFile
    accFile = fd.askopenfilename()


def click_save_file():
    global savingFolder
    savingFolder = fd.askdirectory()
    #print(saving_folder)

def button_open_saved_file_click():
    subprocess.Popen([f"{savingFolder}/{savingFileName.get()}.csv"], shell=True)
    win.destroy()

def rad_select_full_click():
    global fullOrLeft
    fullOrLeft = RadFullOrLeft.get() # return 'full'


def rad_select_left_click():
    global fullOrLeft
    fullOrLeft = RadFullOrLeft.get() # return 'left'

def return_policy(soDonHachToan, soDonDoiTru, soDon, taiKhoanNo, taiKhoanCo):
    if len(soDonHachToan) < 2 and len(soDonDoiTru) < 2:
        return soDon
    elif len(soDonHachToan) < 2 and len(soDon) < 2:
        return soDonDoiTru
    elif len(soDon) < 2 and len(soDonDoiTru) < 2:
        return soDonHachToan
    elif taiKhoanCo.startswith('112'): # TK no -> 'So don DT', TK co -> 'So don HT'
        if len(soDonDoiTru)>2:
            return soDonDoiTru
        else:
            return soDon
    elif taiKhoanNo.startswith('112'): # TK no -> 'So don DT', TK co -> 'So don HT'
        if len(soDonHachToan)>2:
            return soDonHachToan
        else:
            return soDon
    else:
        return


def return_account(taiKhoanNo,taiKhoanCo):
    if taiKhoanNo.startswith('112'):
        return taiKhoanNo
    elif taiKhoanCo.startswith('112'):
        return taiKhoanCo
    else:
        return 'TaiKhoanKhac'


def return_account_du(taiKhoanNo,taiKhoanCo):
    if taiKhoanNo.startswith('112'):
        return taiKhoanCo
    elif taiKhoanCo.startswith('112'):
        return taiKhoanNo
    else:
        return 'TaiKhoanKhac'

def return_amount(taiKHoanNo,taiKhoanCo,soTien):
    if taiKHoanNo.startswith('112'):
        return soTien
    elif taiKhoanCo.startswith('112'):
        return -soTien
    else:
        return 0



def click_execute():
    from tkinter import messagebox as msg
    global dfAccFinal
    global dfPaidListFinal
    global dfPaidList
    global dfAcc
    global dfPaidListGroupBy
    global dfAccGroupByVoucher
    global dfAccFinalVoucher
    global dfMerge
    global paidListFile
    global accFile
    global savingFolder
    global savingFileName

    #name=['No','Paid dated','Client Name','Amount','Currency','Type','Bank/Cash','Note','Số tiền','Place number','Số hồ sơ BT/Số đơn','Kỳ TT','Mã KH','Tài khoản']
    try:
        if paidListFile.endswith('xlsb'):
            dfPaidList = pd.read_excel(paidListFile, usecols=paidListFileUseCol,sheet_name='Claim paid', skiprows=1,engine='pyxlsb')
        else:
            dfPaidList = pd.read_excel(paidListFile, usecols=paidListFileUseCol,sheet_name='Claim paid', skiprows=1)
                  
        if accFile.endswith('xlsb'):
            dfAcc = pd.read_excel(accFile,usecols=accFileUseCol,skiprows=6,dtype=accFileUseColDataType, engine='pyxlsb')
        else:
            dfAcc = pd.read_excel(accFile,usecols=accFileUseCol,skiprows=6,dtype=accFileUseColDataType)
    except:
        msg.showerror("Error", "Only Excel files or CSV files are supported")
        pass

    values={'Số đơn':'0', 'Số đơn HT':'0','Số đơn ĐT':'0'}
    dfAcc=dfAcc.fillna(value=values)
    dfAcc=dfAcc.loc[dfAcc['TK Có'].str.startswith('112')]
    dfAcc['Policy']=dfAcc.apply(lambda x: return_policy(x['Số đơn'], x['Số đơn HT'], x['Số đơn ĐT'], x['TK Nợ'], x['TK Có']), axis=1)
    dfAcc['Tai khoan']=dfAcc.apply(lambda x: return_account(x['TK Nợ'], x['TK Có']), axis=1)  
    dfAcc['Tai khoan doi ung']=dfAcc.apply(lambda x: return_account_du(x['TK Nợ'], x['TK Có']), axis=1)  
    dfAcc['So tien NT']=dfAcc.apply(lambda x: return_amount(x['TK Nợ'], x['TK Có'],x['Số tiền']), axis=1) 
    dfAcc['So tien VND']=dfAcc.apply(lambda x: return_amount(x['TK Nợ'], x['TK Có'],x['Số tiền VND']), axis=1) 
    dfAccFinal=dfAcc.loc[(dfAcc['Tai khoan']!='TaiKhoanKhac') & (dfAcc['Tai khoan doi ung'].str.startswith(('624','331','333','131','133','138','338'))),['Ngày chứng từ','Policy','Loại tiền','So tien NT']] 
    dfAccFinalGroupby=dfAccFinal.groupby(['Ngày chứng từ','Policy','Loại tiền'],as_index=False)['So tien NT'].sum()
    dfAccFinalVoucher=dfAcc.loc[(dfAcc['Tai khoan']!='TaiKhoanKhac') & (dfAcc['Tai khoan doi ung'].str.startswith(('624','331','333','131','133','138','338'))),['Ngày chứng từ','Policy','Loại tiền','Số chứng từ']]
    dfAccGroupByVoucher=dfAccFinalVoucher.groupby(['Ngày chứng từ','Policy','Loại tiền'],as_index=False)['Số chứng từ'].transform(lambda x: ','.join(x))
    
    dfPaidList=dfPaidList[dfPaidList['Số hồ sơ BT/Số đơn'].notna()]
    a=np.datetime64('2000-01-01')
    values={'Paid dated':a ,'Số hồ sơ BT/Số đơn': '0','Currency':'0','Số tiền':0}
    dfPaidList=dfPaidList.fillna(value=values)
    dfPaidListGroupBy=dfPaidList.groupby(['Paid dated','Số hồ sơ BT/Số đơn','Currency'],as_index=False)['Số tiền'].sum()
    if fullOrLeft=='full':
        dfMerge=dfPaidListGroupBy.merge(dfAccFinalGroupby,how='outer',left_on=['Paid dated','Số hồ sơ BT/Số đơn','Currency'],right_on=['Ngày chứng từ','Policy','Loại tiền'],suffixes=['PL','ACC'])
    else:
        dfMerge=dfPaidListGroupBy.merge(dfAccFinalGroupby,how='left',left_on=['Paid dated','Số hồ sơ BT/Số đơn','Currency'],right_on=['Ngày chứng từ','Policy','Loại tiền'],suffixes=['PL','ACC'])
    dfMerge['Chenh lech']=dfMerge['Số tiền']+dfMerge['So tien NT']
    dfMerge.to_csv(f"{savingFolder}/{savingFileName.get()}.csv", encoding="utf-8-sig", index=False)
    msg.showinfo("Information", "Đã hoàn thành")
    subprocess.Popen([f"{savingFolder}/{savingFileName.get()}.csv"], shell=True)
    
    paidListFile = ''
    accFile = ''
    savingFolder = ''
    savingFileName=''
    #radPremiumOrOthers=''
    #dfPaidList=pd.DataFrame
    #dfAcc=pd.DataFrame
    #dfAccFinal=pd.DataFrame
    #dfPaidListFinal=pd.DataFrame
    #dfPaidListGroupBy=pd.DataFrame


# ----------------------------------------------------------------- Widget------------------------------------------------

RadFullOrLeft = tk.StringVar()


rad_select_full = ttk.Radiobutton(win, text='Full', width=30, value='full', variable=RadFullOrLeft, command=rad_select_full_click)
rad_select_full.grid(column=0, row=0)

rad_select_claim = ttk.Radiobutton(win, text='Left', width=30, value='left', variable=RadFullOrLeft, command=rad_select_left_click)
rad_select_claim.grid(column=1, row=0)

ttk.Label(win, text="Chọn file Paid list", width=20).grid(column=0, row=1)
button1 = ttk.Button(win, text="Paid list", width=20, command=click_select_cpc)
button1.grid(column=1, row=1)

ttk.Label(win, text="Chọn file CTGS", width=20).grid(column=0, row=2)
button2 = ttk.Button(win, text="CTGS", width=20, command=click_select_acc)
button2.grid(column=1, row=2)

ttk.Label(win, text="Chọn thư mục lưu file", width=20).grid(column=0, row=3)
button2 = ttk.Button(win, text="Folder", width=20, command=click_save_file)
button2.grid(column=1, row=3)

ttk.Label(win, text="Nhập tên file", width=20).grid(column=0, row=4)
savingFileName = ttk.Entry(win, width=20)
savingFileName.grid(column=1, row=4)

ttk.Label(win, text="Xử lý số liệu", width=20).grid(column=0, row=5)
button3 = ttk.Button(win, text="Run", width=20, command=click_execute)
button3.grid(column=1, row=5)

label_open_saved_file = ttk.Label(win, text="Mở file", width=20).grid(column=0, row=6)
button_open_saved_file = ttk.Button(win, text="Mở file", width=20, command=button_open_saved_file_click)
button_open_saved_file.grid(column=1, row=6)




win.mainloop()

In [149]:
dfAccFinalVoucher

,Ngày chứng từ,Policy,Loại tiền,Số chứng từ
2,2020-09-29,VND49.901.801,VND,04.09_20.0128
5,2020-09-28,TT TIEN MUA QUA TRUNG THU INV 0049465,VND,04.09_20.0154
9,2020-09-29,0,VND,04.09_20.0163
10,2020-09-29,350.435.266VND,VND,04.09_20.0166
189,2020-09-29,0,VND,30141.09_20.0701
...,...,...,...,...
1523,2020-09-28,PAR180005HBF.C.CL01.BT.4,VND,603.09_20.0002
1525,2020-09-28,WCI180002HYD_HOYA.004.CL01.BT.6,VND,605.09_20.0046
1526,2020-09-28,WCI200002HYD_HOYA.001.CL29.BT.2,VND,605.09_20.0047
1527,2020-09-28,WCI200002HYD_HOYA.001.CL22.BT.2,VND,605.09_20.0049


In [148]:
dfAccGroupByVoucher.drop_duplicates()

,Số chứng từ
2,04.09_20.0128
5,04.09_20.0154
9,"04.09_20.0163,30141.09_20.0701,30141.09_20.070..."
10,04.09_20.0166
198,"30141.09_20.0702,30141.09_20.0708"
268,30141.09_20.0747
357,30141.09_20.0764
412,"30141.09_20.0846,30141.09_20.0861"
416,30141.09_20.0848
428,"30141.09_20.0866,30141.09_20.0866,30141.09_20...."


In [147]:
dfAccFinalGroupby

,Ngày chứng từ,Policy,Loại tiền,So tien NT
0,2020-01-10,CPH180010SGD.CL36.BT.2,VND,-11651327.0
1,2020-01-10,CPH180013HBD.CL38.BT.2,VND,-3500000.0
2,2020-01-10,CPH180013SGD.CL03.BT.2,VND,-2513000.0
3,2020-01-10,CPH180059HPD.CL35.BT.2,VND,-200000.0
4,2020-01-10,CPH180068HPD.CL32.BT.2,VND,-750000.0
...,...,...,...,...
4182,2020-08-27,Inv0009094_384.878,VND,-349889.0
4183,2020-08-27,Inv0013164_3.615.000,VND,-3286364.0
4184,2020-08-27,Inv0021533_326.700,VND,-297000.0
4185,2020-08-27,Inv0027682_1.358.500,VND,-1235000.0


In [150]:
dfAccFinalVoucher['So chung tu']=dfAccFinalVoucher.groupby(['Ngày chứng từ','Policy','Loại tiền'],as_index=False)['Số chứng từ'].transform(lambda x: ','.join(x))
dfAccGroupByVoucher=dfAccFinalVoucher.drop_duplicates()

In [151]:
dfAccGroupByVoucher

,Ngày chứng từ,Policy,Loại tiền,Số chứng từ,So chung tu
2,2020-09-29,VND49.901.801,VND,04.09_20.0128,04.09_20.0128
5,2020-09-28,TT TIEN MUA QUA TRUNG THU INV 0049465,VND,04.09_20.0154,04.09_20.0154
9,2020-09-29,0,VND,04.09_20.0163,"04.09_20.0163,30141.09_20.0701,30141.09_20.070..."
10,2020-09-29,350.435.266VND,VND,04.09_20.0166,04.09_20.0166
189,2020-09-29,0,VND,30141.09_20.0701,"04.09_20.0163,30141.09_20.0701,30141.09_20.070..."
...,...,...,...,...,...
1523,2020-09-28,PAR180005HBF.C.CL01.BT.4,VND,603.09_20.0002,603.09_20.0002
1525,2020-09-28,WCI180002HYD_HOYA.004.CL01.BT.6,VND,605.09_20.0046,605.09_20.0046
1526,2020-09-28,WCI200002HYD_HOYA.001.CL29.BT.2,VND,605.09_20.0047,605.09_20.0047
1527,2020-09-28,WCI200002HYD_HOYA.001.CL22.BT.2,VND,605.09_20.0049,605.09_20.0049
